In [2]:
import torch
import torch.nn as nn

In [5]:
L = 5
dim = 3
query = torch.randn(1,L, dim)
key = torch.randn(1,L, dim)
value = torch.randn(1, L, dim)
rel_pos_emb = torch.randn(2*L-1, dim)

In [10]:
seq_pos = torch.arange(L)
seq_pos[None, :] - seq_pos[:, None]

tensor([[ 0,  1,  2,  3,  4],
        [-1,  0,  1,  2,  3],
        [-2, -1,  0,  1,  2],
        [-3, -2, -1,  0,  1],
        [-4, -3, -2, -1,  0]])

In [26]:
rel_pos = seq_pos[:,None] - seq_pos[None,:] # (L, L)
rel_pos = seq_pos[None, :] - seq_pos[:, None]
rel_pos

tensor([[ 0,  1,  2,  3,  4],
        [-1,  0,  1,  2,  3],
        [-2, -1,  0,  1,  2],
        [-3, -2, -1,  0,  1],
        [-4, -3, -2, -1,  0]])

In [27]:
rel_pos2 = rel_pos + L -1 # convert all the negative indices to positive indices
rel_pos2

tensor([[4, 5, 6, 7, 8],
        [3, 4, 5, 6, 7],
        [2, 3, 4, 5, 6],
        [1, 2, 3, 4, 5],
        [0, 1, 2, 3, 4]])

In [186]:
2*L -1

9

In [34]:
rel_pos2

tensor([[4, 5, 6, 7, 8],
        [3, 4, 5, 6, 7],
        [2, 3, 4, 5, 6],
        [1, 2, 3, 4, 5],
        [0, 1, 2, 3, 4]])

In [56]:
rel_pos_emb_pos = rel_pos_emb[rel_pos2]  # (L, D)
rel_pos_emb_pos.shape

torch.Size([5, 5, 3])

In [60]:
rel_pos.shape, rel_pos_emb.shape

(torch.Size([5, 5]), torch.Size([9, 3]))

In [62]:
rel_pos2

tensor([[4, 5, 6, 7, 8],
        [3, 4, 5, 6, 7],
        [2, 3, 4, 5, 6],
        [1, 2, 3, 4, 5],
        [0, 1, 2, 3, 4]])

In [65]:
rel_pos_emb

tensor([[ 1.0906,  0.3163, -1.4240],
        [-1.0363,  0.7225,  0.9131],
        [-0.9518,  0.1109, -0.5306],
        [ 0.7835, -0.6042,  0.4329],
        [ 0.0445,  0.7216, -1.0556],
        [-0.1353,  0.7443,  1.8683],
        [ 1.0703,  1.8171,  0.4764],
        [-0.3758,  0.0659, -1.3575],
        [ 1.4050, -0.8572,  0.4682]])

In [100]:
rel_pos2.tolist()

([4, 5, 6, 7, 8],
 [3, 4, 5, 6, 7],
 [2, 3, 4, 5, 6],
 [1, 2, 3, 4, 5],
 [0, 1, 2, 3, 4])

In [88]:
rel_pos2.unsqueeze(-1).shape, rel_pos_emb.unsqueeze(1).shape

(torch.Size([5, 5, 1]), torch.Size([9, 1, 3]))

In [193]:
rel_pos_emb_final = rel_pos_emb[rel_pos2] # (L, L, D)
# query.matmul(rel_pos_emb_final.transpose(2,0).transpose(1,2).unsqueeze(0))
rel_pos_emb_final.shape

torch.Size([5, 5, 3])

In [ ]:
# (B, 1, D), (T, D)

In [148]:
# verify that the attention score is correct
query[:,0:1,:].matmul(rel_pos_emb[[4,5,6,7,8],:].unsqueeze(0).transpose(1,2))

tensor([[[ 0.1204, -1.2159,  0.3717, -0.0092,  2.0377]]])

In [175]:
# relative attention score
# (B,T,1,D), (1,T,D,T) -> (B, T, 1, T) -> (B, T, T)
torch.matmul(query.unsqueeze(2), rel_pos_emb_final.unsqueeze(0).transpose(-1,-2)).squeeze(-2) # (B, T, T)
# attention_scores = attention_scores + rel_pos_scores  # Add to attention scores

tensor([[[ 0.1204, -1.2159,  0.3717, -0.0092,  2.0377],
         [-0.2955, -0.8731, -0.5363, -3.2205,  0.3245],
         [-0.2409,  0.3046,  0.6400, -1.5759, -0.0343],
         [-1.8551, -1.1569,  1.5205, -0.8030, -0.8787],
         [ 3.9724, -2.9123,  0.5911, -0.3976,  2.3682]]])

In [203]:
5*4*3 / (3*2*1) # Each unique group of 2 fruits has $2! = 2$ possible orders.


10.0

In [201]:
count = 0
for i in range(5):
    for j in range(i+1,5):
        for k in range(j+1, 5):
            print(i, j, k)
            count+=1
count

0 1 2
0 1 3
0 1 4
0 2 3
0 2 4
0 3 4
1 2 3
1 2 4
1 3 4
2 3 4


10

In [179]:
# positional_embedding = torch.nn.Parameter(torch.randn(2 * L - 1, dim))
positional_embedding = rel_pos_emb
print(positional_embedding.shape) # torch.Size([9, 11])

# X can be the Query or Key vectors of shape (B, T, D)
relative_positional_encoding = torch.matmul(query, positional_embedding.transpose(0,1)) # B, T, 2*T-1)
print(relative_positional_encoding.shape) # # torch.Size([2, 5, 9])

torch.Size([9, 3])
torch.Size([1, 5, 9])


In [194]:
relative_positional_encoding[0,1,-L-1:] # (B, T, T)

tensor([-0.2955, -0.8731, -0.5363, -3.2205,  0.3245, -0.7854])

In [158]:
query[:,0:1,:].repeat(1,1,5,1).matmul(rel_pos_emb[[4,5,6,7,8],:].unsqueeze(0).unsqueeze(0)) # (B, 1, 1, D) (T, D)

RuntimeError: Expected size for first two dimensions of batch2 tensor to be: [1, 3] but got: [1, 5].

In [163]:
rel_pos_emb_final.shape # (T, T, D)

torch.Size([5, 5, 3])

In [164]:
query.shape

torch.Size([1, 5, 3])

In [173]:
torch.matmul(query.unsqueeze(2), rel_pos_emb_final.unsqueeze(0).transpose(-1,-2)).shape  # (B,T,D), (T,T, D) -> (batch_size, L, L)

torch.Size([1, 5, 1, 5])

In [174]:
torch.matmul(query.unsqueeze(2), rel_pos_emb_final.unsqueeze(0).transpose(-1,-2)).squeeze(-2)

tensor([[[ 0.1204, -1.2159,  0.3717, -0.0092,  2.0377],
         [-0.2955, -0.8731, -0.5363, -3.2205,  0.3245],
         [-0.2409,  0.3046,  0.6400, -1.5759, -0.0343],
         [-1.8551, -1.1569,  1.5205, -0.8030, -0.8787],
         [ 3.9724, -2.9123,  0.5911, -0.3976,  2.3682]]])

In [159]:
query[:,0:1,:].repeat(1,1,5,1).shape

torch.Size([1, 1, 5, 3])

In [157]:
query[:,0:1,:].repeat(1,1,5,1).shape

torch.Size([1, 1, 5, 3])

In [154]:
rel_pos_emb[[4,5,6,7,8],:].unsqueeze(0).unsqueeze(0).shape

torch.Size([1, 1, 5, 3])

In [ ]:
# (B, T, D), (T,T,D) -> (B, T, T)

In [130]:
# (B, T,1,D), (1, T, T,D) -> (B, T, T, D)
(query.unsqueeze(2) * rel_pos_emb_final.unsqueeze(0)).sum(dim=-2) # (B,T, D)


tensor([[[ 2.6000, -1.1458, -0.1495],
         [-1.6982, -2.9202,  0.0176],
         [ 0.5109, -0.5625, -0.8548],
         [-1.3728, -1.8783,  0.0779],
         [-0.0455, -0.1973,  3.8646]]])

In [127]:
rel_pos_emb[[4,5,6,7,8],:] # (T, D)
query[:,0:1,:].matmul(rel_pos_emb[[4,5,6,7,8],:].unsqueeze(0).transpose(1,2)) # (B, 1, D), (1, T, D)

tensor([[[ 0.1204, -1.2159,  0.3717, -0.0092,  2.0377]]])

In [137]:
rel_pos_emb[[4,5,6,7,8],:].shape, query[:,0:1,:].shape

(torch.Size([5, 3]), torch.Size([1, 1, 3]))

In [132]:
rel_pos_emb[[4,5,6,7,8],:].unsqueeze(0).transpose(1,2).shape

torch.Size([1, 3, 5])

In [125]:
query[:,0:1,:].shape

torch.Size([1, 1, 3])

In [126]:
rel_pos_emb[[4,5,6,7,8],:].unsqueeze(0).shape

torch.Size([1, 5, 3])

In [57]:
torch.batch_rel_pos_emb_pos.shape

torch.Size([5, 5, 3])

In [53]:
rel_pos2.shape

torch.Size([5, 5])

In [40]:
rel_pos_emb.shape

torch.Size([9, 3])

In [38]:
rel_pos.shape

torch.Size([5, 5])

In [ ]:
# goal of attention: (B,T,D) -> B, T, D
# goal of rel_pos: similarity + rel_position_context # (B, T, D)

In [ ]:
query @ rel_pos_emb_pos # (B, L, D), (L, L, D)

In [19]:
# for r(i,j) = r(i-j+T-1)

torch.Size([9, 3])

In [224]:
corpus = [
    "This is the Hugging Face Course.",
    "This chapter is about tokenization.",
    "This section shows several tokenizer algorithms.",
    "Hopefully, you will be able to understand how they are trained and generate tokens.",
]
from collections import defaultdict
word_freqs = defaultdict(int)
for text in corpus:
    tokens = text.split()
    for token in tokens:
        word_freqs[token] += 1

alphabets = []
for word in word_freqs:
    for letter in word:
        if letter not in alphabets:
            alphabets.append(letter)
alphabets.sort()

vocab = [ '<|endoftext|>'  ]+ alphabets.copy()

splits = {word: [c for c in word] for word in word_freqs.keys()}

def compute_pair_freqs(splits):
    pair_freqs = defaultdict(int)
    for word, freq in word_freqs.items():
        split = splits[word]
        if len(split) == 1:
            continue
        for i in range(len(split) - 1):
            pair = (split[i], split[i + 1])
            pair_freqs[pair] += freq
    return pair_freqs

pair_freqs = compute_pair_freqs(splits)

best_pair = ""
max_freq = None

for pair, freq in pair_freqs.items():
    if max_freq is None or max_freq < freq:
        best_pair = pair
        max_freq = freq

print(best_pair, max_freq)

merges = {("i", "s"): "is"}
vocab.append("is")

def merge_pair(a, b, splits):
    for word in word_freqs:
        split = splits[word]
        if len(split) == 1:
            continue

        i = 0
        while i < len(split) - 1:
            if split[i] == a and split[i + 1] == b:
                split = split[:i] + [a + b] + split[i + 2 :]
            else:
                i += 1
        splits[word] = split
    return splits


splits = merge_pair("i", "s", splits)
print(splits["This"])


vocab_size = 50

while len(vocab) < vocab_size:
    pair_freqs = compute_pair_freqs(splits)
    best_pair = ""
    max_freq = None
    for pair, freq in pair_freqs.items():
        if max_freq is None or max_freq < freq:
            best_pair = pair
            max_freq = freq
    splits = merge_pair(*best_pair, splits)
    merges[best_pair] = best_pair[0] + best_pair[1]
    vocab.append(best_pair[0] + best_pair[1])

    
def tokenize(text):
    pre_tokenized_text = text.split()
    splits = [[l for l in word] for word in pre_tokenized_text]
    for pair, merge in merges.items():
        for idx, split in enumerate(splits):
            i = 0
            while i < len(split) - 1:
                if split[i] == pair[0] and split[i + 1] == pair[1]:
                    split = split[:i] + [merge] + split[i + 2 :]
                else:
                    i += 1
            splits[idx] = split

    return sum(splits, [])

tokenize("This is the Hugging Face Course. tokenizer")


('i', 's') 5
['T', 'h', 'is']


['This',
 'is',
 'the',
 'H',
 'u',
 'g',
 'g',
 'in',
 'g',
 'F',
 'a',
 'c',
 'e',
 'C',
 'ou',
 'r',
 'se',
 '.',
 'tokeniz',
 'er']

50

In [227]:
pair_freqs

defaultdict(int,
            {('H', 'u'): 1,
             ('u', 'g'): 1,
             ('g', 'g'): 1,
             ('g', 'in'): 1,
             ('in', 'g'): 1,
             ('F', 'a'): 1,
             ('a', 'c'): 1,
             ('c', 'e'): 1,
             ('C', 'ou'): 1,
             ('ou', 'r'): 1,
             ('r', 'se'): 1,
             ('se', '.'): 1,
             ('c', 'h'): 1,
             ('h', 'a'): 1,
             ('a', 'p'): 1,
             ('p', 't'): 1,
             ('t', 'er'): 1,
             ('ab', 'ou'): 1,
             ('ou', 't'): 1,
             ('tokeniz', 'at'): 1,
             ('at', 'io'): 1,
             ('io', 'n'): 2,
             ('n', '.'): 1,
             ('se', 'c'): 1,
             ('c', 't'): 1,
             ('t', 'io'): 1,
             ('s', 'h'): 1,
             ('h', 'o'): 2,
             ('o', 'w'): 2,
             ('w', 's'): 1,
             ('se', 'v'): 1,
             ('v', 'er'): 1,
             ('er', 'a'): 1,
             ('a', 'l'): 2,
      

In [226]:
l = [ 1,2,3]
l[4:]

[]

In [225]:
splits

{'This': ['This'],
 'is': ['is'],
 'the': ['the'],
 'Hugging': ['H', 'u', 'g', 'g', 'in', 'g'],
 'Face': ['F', 'a', 'c', 'e'],
 'Course.': ['C', 'ou', 'r', 'se', '.'],
 'chapter': ['c', 'h', 'a', 'p', 't', 'er'],
 'about': ['ab', 'ou', 't'],
 'tokenization.': ['tokeniz', 'at', 'ion', '.'],
 'section': ['se', 'c', 't', 'ion'],
 'shows': ['s', 'h', 'o', 'w', 's'],
 'several': ['se', 'v', 'er', 'a', 'l'],
 'tokenizer': ['tokeniz', 'er'],
 'algorithms.': ['a', 'l', 'g', 'o', 'r', 'i', 'th', 'm', 's', '.'],
 'Hopefully,': ['H', 'o', 'p', 'e', 'f', 'u', 'l', 'l', 'y', ','],
 'you': ['y', 'ou'],
 'will': ['w', 'i', 'l', 'l'],
 'be': ['b', 'e'],
 'able': ['ab', 'l', 'e'],
 'to': ['to'],
 'understand': ['u', 'nd', 'er', 's', 't', 'a', 'nd'],
 'how': ['h', 'o', 'w'],
 'they': ['the', 'y'],
 'are': ['a', 'r', 'e'],
 'trained': ['t', 'r', 'a', 'in', 'e', 'd'],
 'and': ['a', 'nd'],
 'generate': ['g', 'en', 'er', 'at', 'e'],
 'tokens.': ['token', 's', '.']}

In [222]:
merges

{('i', 's'): 'is',
 ('e', 'r'): 'er',
 ('t', 'o'): 'to',
 ('e', 'n'): 'en',
 ('T', 'h'): 'Th',
 ('Th', 'is'): 'This',
 ('t', 'h'): 'th',
 ('o', 'u'): 'ou',
 ('s', 'e'): 'se',
 ('to', 'k'): 'tok',
 ('tok', 'en'): 'token',
 ('n', 'd'): 'nd',
 ('th', 'e'): 'the',
 ('i', 'n'): 'in',
 ('a', 'b'): 'ab',
 ('token', 'i'): 'tokeni',
 ('tokeni', 'z'): 'tokeniz',
 ('a', 't'): 'at',
 ('i', 'o'): 'io',
 ('io', 'n'): 'ion'}

('i', 's')